In [111]:
from IPython.display import clear_output
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [112]:
# Initialisation des variables
X_train = np.array([])
Y_train = np.array([])
X_test = np.array([])
Y_test = np.array([])
X_validation = np.array([])
Y_validation = np.array([])
old_type_data = "dataset"

In [113]:
# Transformation des photos en dataframe
def photo_to_dataframe(chemins_photos: str, label: str, separation: str, type_data: str):
    """Transforme les photos en tableau de pixels et les sauvegarde dans un fichier csv

    Args:
        chemins_photos (str): Chemin des photos (on veut le dossier contenant les photos)
        label (str): label de la photo (ce qui va être prédit)
        separation (str): séparation des données (train, test, validation)
    """
    # Initialisation des variables globales dans la fonction
    global X_train
    global Y_train
    global X_test
    global Y_test
    global X_validation
    global Y_validation
    global old_type_data
    
    print("..\Dataset")
    print("    \\" + type_data)
    print("      \\" + separation)
    print("        \\" + label)# Afficher le dossier en cours de traitement
    # Parcours et transformation des photos avec barre de progression 
    with tqdm(total=len(chemins_photos), desc="Proccessing" , position=0, leave=True) as pbar:
        for chemin in chemins_photos:
            
            pbar.set_postfix(img=chemin[-10:]) # Mettre à jour la barre de progression avec des informations supplémentaires
            
            img = Image.open(chemin) # 300 x 300
            nouvelle_taille = (50, 50) # 2500 
            img_redimensionnee = img.resize(nouvelle_taille) # Redimensionner l'image
            nb_image = img_redimensionnee.convert('L') # Convertion en noir et blanc
            
            #  Récupération de la liste des pixels
            tab = []
            for i in range(nb_image.size[1]):
                row = []
                for y in range(nb_image.size[0]):
                    # Récupérer la couleur du pixel
                    row.append(nb_image.getpixel((y, i)))
                # tab.append(np.array(row))
                tab.append(row)
                
            tab_numpy = np.array(tab) / 255 # mise à l'échelle des données
            
            # Sauvegarde des données dans un fichier npz
            if old_type_data != type_data: # si ils ne sont pls égaux, c'est qu'on a changé de type de données (dataset, datasetDigit, datasetSmall)
                # sauvegarde des tableaux numpy
                np.savez("../Dataset/" + old_type_data + ".npz", tableau1=X_train, tableau2=Y_train, tableau3=X_test, tableau4=Y_test, tableau5=X_validation, tableau6=Y_validation)
                
                # Réinitialisation des variables
                X_train = np.array([])
                Y_train = np.array([])
                X_test = np.array([])
                Y_test = np.array([])
                X_validation = np.array([])
                Y_validation = np.array([])
                
            if separation == "test":
                X_test = np.append(X_test, tab_numpy)
                Y_test = np.append(Y_test, label)
            elif separation == "train":
                X_train = np.append(X_train, tab_numpy)
                Y_train = np.append(Y_train, label)
            elif separation == "validation":
                X_validation = np.append(X_validation, tab_numpy)
                Y_validation = np.append(Y_validation, label)
            
            old_type_data = type_data
                
            pbar.update(1) # Mettre à jour la barre de progression
    
    clear_output()  # Effacer l'output précédent (sinon pb d'affichage de la barre de progression)

In [114]:
# Parcours des dossiers et sous-dossiers pour recréer le dataset (recursion)
def recreate_dataset(chemin: str, profondeur: int, label: str, separation: str, type_data: str):
    """Réecriture du dataset: transformation des photos en tableau de pixels et les sauvegarde dans un fichier csv
    Cette fonction est récursive et doit être executer sur un dossier contenant la même arborescence que le dossier ../Dataset

    Args:
        chemin (str): chemin du dataset
        profondeur (int): profondeur de la recursion (elle permet aussi de récupérer les labels et les séparations)
        label (str): label de la photo (ce qui va être prédit)
        separation (str): séparation des données (train, test, validation)
    """
    dossiers = [f for f in os.listdir(chemin) if os.path.isdir(os.path.join(chemin, f))]
    if len(dossiers) == 0: # Si aucun dossier n'est trouvé, on est dans le cas de base
        fichiers_png = [os.path.join(chemin, f) for f in os.listdir(chemin) if f.endswith('.png')] # on recupère et stocke les fichiers .png
        photo_to_dataframe(fichiers_png, label, separation, type_data) # On transforme les photos en tableau en df qu'on save dans un fichier.csv
    else : # Sinon, on continue de parcourir les dossiers
        for dossier in dossiers:
            if type_data == 0:
                type_data = dossier
            if profondeur == 1:
                separation = dossier
            if profondeur == 2:
                label = dossier
            recreate_dataset(os.path.join(chemin, dossier), profondeur + 1, label, separation, type_data)

In [115]:
# Appel de la fonction de parcours des dossiers
recreate_dataset("..\Dataset", 0, "truc", "bidule", "dataset")
# La valeur "test" comme 5ème argument est obligatoire pour ne pas avoir de fichier en plus ...

..\Dataset
    \dataset
      \test
        \E


Proccessing:  28%|██▊       | 349/1263 [00:08<00:22, 41.18it/s, img=000349.png]


KeyboardInterrupt: 